In [42]:
import os
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
X_train_full.shape

In [ ]:
X_train_full.dtype

In [11]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

In [ ]:
plt.imshow(X_train[0], cmap="binary")
plt.axis("off")
plt.show()

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis("off")
        plt.title(y_train[index], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [31]:
K = keras.backend


# Callback personalizado
class ExponentialLearningRate(keras.callbacks.Callback):
    def __init__(self, factor):
        super().__init__()
        self.factor = factor
        self.rates = []
        self.losses = []

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        self.losses.append(logs.get("loss", None))

        # Obtener el valor de learning_rate y asegurarnos de que sea un número
        lr = self.model.optimizer.learning_rate
        if isinstance(lr, tf.Variable):
            lr = lr.numpy()  # Convertir a valor numérico si es un tf.Variable

        # Registrar la tasa de aprendizaje
        self.rates.append(lr)

        # Actualizar el learning rate
        if isinstance(lr, (float, int)):
            new_lr = lr * self.factor
            self.model.optimizer.learning_rate.assign(new_lr)  # Modificar learning_rate

In [32]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [33]:
model = keras.models.Sequential(
    [
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax"),
    ]
)

In [34]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=["accuracy"],
)
expon_lr = ExponentialLearningRate(factor=1.005)

In [ ]:
history = model.fit(
    X_train, y_train, epochs=1, validation_data=(X_valid, y_valid), callbacks=[expon_lr]
)

In [ ]:
plt.plot(expon_lr.rates, expon_lr.losses)
plt.gca().set_xscale("log")
plt.hlines(min(expon_lr.losses), min(expon_lr.rates), max(expon_lr.rates))
plt.axis([min(expon_lr.rates), max(expon_lr.rates), 0, expon_lr.losses[0]])
plt.grid()
plt.xlabel("Learning rate")
plt.ylabel("Loss")

In [38]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [39]:
model = keras.models.Sequential(
    [
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax"),
    ]
)

In [40]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=3e-1),
    metrics=["accuracy"],
)

In [ ]:
run_index = 1  # increment this at every run
run_logdir = os.path.join(os.curdir, "my_mnist_logs", "run_{:03d}".format(run_index))
run_logdir

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "my_mnist_model.keras", save_best_only=True
)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

history = model.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb],
)

In [ ]:
model = keras.models.load_model("my_mnist_model.keras")  # rollback to best model
model.evaluate(X_test, y_test)

In [ ]:
%tensorboard --logdir=./my_mnist_logs --port=6006